In [1]:
from face_detector import face_alignment
from feature_extractor.feature_extractor import fitur

import numpy as np
import pandas as pd
import io
import base64

import torchvision.datasets as datasets
import torchvision.transforms as transforms
from PIL import Image

from torch.utils.data import DataLoader
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt

import csv
import os

face_detect  = False

output_path = "/"

/Users/taufiq/infran-spark/feature_extractor/feature_extractor.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  backbone.load_state_dict(torch.load(model_path, map_locat

In [ ]:
a = np.zeros([1, 10])
print(a)
# a.dumps()

base64_images = []

def pil_collate_fn(batch):
    # This will be a list of tuples (image, label)
    images, labels = zip(*batch)
    return list(images), labels

In [ ]:
transform_image = datasets.ImageFolder("AgeDB/imgs_agedb")
dataloader = DataLoader(transform_image, batch_size=1, shuffle=False, collate_fn=pil_collate_fn)

class_names = transform_image.classes
label_1 = []
file_names = []  # List to store file names

# Prepare to store images if needed
base64_images = []  # If you are handling images

for idx, (img, label) in enumerate(dataloader):
    img_pil = img[0]
    label_name = class_names[label[0]]
      # Store image if needed
    label_1.append(label_name)
    buffered = io.BytesIO()
    img_pil.save(buffered, format="JPEG")
    img_base64 = base64.b64encode(buffered.getvalue())

    base64_images.append(img_base64)

    # Get the file path
    file_path, _ = transform_image.samples[idx]
    file_name = os.path.basename(file_path)

    file_names.append(file_name)  # Corrected to append file_name

# print(len(file_names))
print(len(class_names))




In [ ]:
# Detect faces
face_detect_list = []
for img_string in base64_images: 
    x = face_alignment(img_string)
    face_detect_list.append(x)
# print(face_detect_list[0])

# Extract features
matrix_embedding = []
for face_align in face_detect_list:
    y = fitur(face_align, face_detect)
    embedding_feature = np.load(io.BytesIO(y), allow_pickle=True)
    embedding_feature = list(pd.DataFrame(embedding_feature).values[0])
    matrix_embedding.append(embedding_feature)

matrix_embedding = np.array(matrix_embedding)

# Calculate similarity
similarity = cosine_similarity(matrix_embedding)
similarity = similarity.clip(min=0, max=1)

In [ ]:
csv_file_path = 'output_with_vertical_similarity_fulllfw.csv'  # Specify your desired CSV file path

with open(csv_file_path, mode='w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write header
    csv_writer.writerow(['File Name', 'Compared To', 'Similarity Score'])
    
    # Write data rows
    for idx in range(len(file_names)):
        for compared_idx in range(len(file_names)):
            # Include self-comparison by removing the condition that excludes it
            row = [
                file_names[idx],  
                file_names[compared_idx], 
                similarity[idx, compared_idx]
            ]
            csv_writer.writerow(row)

In [4]:
# Face Detect 
with open("2325_AbeVigoda_49_m.jpg","rb") as image_file: 
    encoded_string = base64.b64encode(image_file.read())


face_crop = face_alignment(encoded_string)
# face_crop

extract = fitur(face_crop, face_detect)
extract_embedding = np.load(io.BytesIO(extract), allow_pickle=True)
extract_embedding = list(pd.DataFrame(extract_embedding).values[0])
extract_embedding

[np.float64(0.006806168705224991),
 np.float64(-0.04703041538596153),
 np.float64(-0.008280658163130283),
 np.float64(0.009128457866609097),
 np.float64(-0.09598100185394287),
 np.float64(-0.05394572764635086),
 np.float64(0.008444173261523247),
 np.float64(0.024636702612042427),
 np.float64(-0.08524149656295776),
 np.float64(0.014375751838088036),
 np.float64(-0.03795165196061134),
 np.float64(0.023857250809669495),
 np.float64(0.008998426608741283),
 np.float64(0.042824067175388336),
 np.float64(0.03794248774647713),
 np.float64(0.04334882274270058),
 np.float64(-0.01229157391935587),
 np.float64(-0.03545321151614189),
 np.float64(-0.014606449753046036),
 np.float64(-0.02048022300004959),
 np.float64(0.01875459961593151),
 np.float64(-0.0031242931727319956),
 np.float64(0.005278741475194693),
 np.float64(0.07216653972864151),
 np.float64(-0.026483559980988503),
 np.float64(-0.03144698962569237),
 np.float64(0.059042707085609436),
 np.float64(0.01196744479238987),
 np.float64(0.004985